In [9]:
import mne
import tkinter as tk
from tkinter import filedialog, messagebox
from collections import Counter
"""
  * @author: dengyufeng
  * @Created on 2024/8/14 9:42
 """
# 创建GUI应用程序
root = tk.Tk()
root.title('EEG数据标注检查工具')

# 定义一个函数来加载并检查标注信息
def check_annotations(file_format):
    filetypes = [(f"{file_format.upper()} files", f"*.{file_format}")]
    file_path = filedialog.askopenfilename(filetypes=filetypes)
    
    if file_path:
        try:
            # 根据不同格式加载文件
            if file_format == 'edf':
                raw = mne.io.read_raw_edf(file_path, preload=True)
            elif file_format == 'bdf':
                raw = mne.io.read_raw_bdf(file_path, preload=True)
            elif file_format == 'fif':
                raw = mne.io.read_raw_fif(file_path, preload=True)
            else:
                raise ValueError("不支持的文件格式")
            
            # 检查标注信息，并按发生时间排序
            annotations = raw.annotations
            if annotations:
                sorted_annotations = sorted(annotations, key=lambda x: x['onset'])
                
                # 统计每种标记的数量
                descriptions = [annot['description'] for annot in sorted_annotations]
                counts = Counter(descriptions)
                
                # 生成展示信息
                lines = []
                
                # 添加标注信息和时间间隔并排显示
                lines.append("标注信息按时间排序（标记 - 时间间隔）:")
                for i in range(len(sorted_annotations)):
                    annot = sorted_annotations[i]
                    # 当前标记描述
                    current_line = f"标记 {i+1}: {annot['description']} (时间={annot['onset']:.2f}秒)"
                    # 如果不是最后一个标记，添加到下一个标记的时间间隔
                    if i < len(sorted_annotations) - 1:
                        interval = sorted_annotations[i + 1]['onset'] - annot['onset']
                        current_line += f" - 间隔: {interval:.2f}秒"
                    lines.append(current_line)
                
                # 添加标记数量信息
                lines.append("\n标记数量:")
                for desc, count in counts.items():
                    lines.append(f"标记 '{desc}': {count} 次")
                
                # 将所有内容拼接为一个字符串
                info_display = "\n".join(lines)
                
                messagebox.showinfo("标注信息", info_display)
            else:
                messagebox.showinfo("结果", "没有标注信息。")
        
        except Exception as e:
            messagebox.showerror("错误", f"加载{file_format.upper()}文件失败: {e}")

# 创建按钮以选择不同格式的EEG文件并检查标注信息
button_edf = tk.Button(root, text="选择EDF文件并检查标注信息", command=lambda: check_annotations('edf'))
button_edf.pack(pady=10)

button_bdf = tk.Button(root, text="选择BDF文件并检查标注信息", command=lambda: check_annotations('bdf'))
button_bdf.pack(pady=10)

button_fif = tk.Button(root, text="选择FIF文件并检查标注信息", command=lambda: check_annotations('fif'))
button_fif.pack(pady=10)

# 添加关闭按钮
close_button = tk.Button(root, text="关闭", command=root.destroy)
close_button.pack(pady=20)

# 运行GUI主循环
root.mainloop()


Extracting EDF parameters from C:\Users\pc\Desktop\李佳\S002\S002R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
